In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.elasticsearch:elasticsearch-spark-30_2.12:8.11.0 pyspark-shell'

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, to_timestamp, when, window, expr
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType

# Créer une session Spark
spark = SparkSession.builder \
    .appName("IoT Kafka") \
    .config("spark.jars.packages",
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,"
            "org.elasticsearch:elasticsearch-spark-30_2.12:8.12.2") \
    .getOrCreate()

# Schéma des données JSON
schema = StructType() \
    .add("device_id", StringType()) \
    .add("temperature", DoubleType()) \
    .add("humidity", DoubleType()) \
    .add("pressure", DoubleType()) \
    .add("vibration", DoubleType()) \
    .add("latitude", DoubleType()) \
    .add("longitude", DoubleType()) \
    .add("timestamp", StringType())

# Lire depuis Kafka
raw_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:9092") \
    .option("subscribe", "iot_raw_data") \
    .option("startingOffsets", "latest") \
    .load()

# Extraire le JSON
parsed_df = raw_df.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

# Conversion du timestamp
parsed_df = parsed_df.withColumn("timestamp", to_timestamp("timestamp"))

# Nettoyage des valeurs manquantes
clean_df = parsed_df.dropna()

# Ajout des règles métiers
alert_df = clean_df.withColumn(
    "status",
    when(col("temperature") > 50, "Alerte")
    .when((col("humidity") < 10) & (col("temperature") > 40), "Critique")
    .when((col("vibration") > 5) & ((col("pressure") < 950) | (col("pressure") > 1050)), "Alerte")
    .otherwise("Normal")
)

# Capteurs inactifs (si aucune donnée depuis >1h) => traitement par une autre logique (batch/join avec état)

# Exemple de corrélation régionale : moyenne température par région (arrondi lat/lon à 0.1°)
regional_alerts = alert_df \
    .withColumn("region_lat", expr("round(latitude, 1)")) \
    .withColumn("region_lon", expr("round(longitude, 1)")) \
    .groupBy(window("timestamp", "10 minutes"), "region_lat", "region_lon") \
    .avg("temperature", "humidity") \
    .withColumnRenamed("avg(temperature)", "avg_temp") \
    .withColumnRenamed("avg(humidity)", "avg_humidity") \
    .withColumn("regional_status", when(col("avg_temp") > 45, "Alerte regionale").otherwise("OK"))

# Écriture dans Elasticsearch
query = alert_df.writeStream \
    .outputMode("append") \
    .format("org.elasticsearch.spark.sql") \
    .option("checkpointLocation", "/tmp/checkpoint_iot") \
    .option("es.nodes", "es01") \
    .option("es.port", "9200") \
    .option("es.nodes.wan.only", "true") \
    .option("es.net.ssl", "true") \
    .option("es.net.ssl.cert.allow.self.signed", "true") \
    .option("es.net.http.auth.user", "elastic") \
    .option("es.net.http.auth.pass", "Eselpil2") \
    .option("es.resource", "agritech_iot_data_enriched/") \
    .start()\
    .awaitTermination()

# Kafka stream visible côté console (debug)
raw_df.selectExpr("CAST(value AS STRING)").writeStream \
    .format("console") \
    .outputMode("append") \
    .start()\
    .awaitTermination()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=56>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline(

Py4JError: An error occurred while calling o109.awaitTermination

In [3]:
from pyspark.sql import Row

sample_data = [Row(device_id="test-device", temperature=42.0, humidity=20.0,
                   pressure=1012.0, vibration=0.5, latitude=14.7, longitude=-17.4,
                   timestamp="2025-04-17T20:00:00")]

df = spark.createDataFrame(sample_data)

df.write \
    .format("org.elasticsearch.spark.sql") \
    .option("es.nodes", "es01") \
    .option("es.port", "9200") \
    .option("es.nodes.wan.only", "true") \
    .option("es.net.ssl", "true") \
    .option("es.net.ssl.cert.allow.self.signed", "true") \
    .option("es.net.http.auth.user", "elastic") \
    .option("es.net.http.auth.pass", "Eselpil2") \
    .option("es.resource", "agritech_iot_test/") \
    .mode("overwrite") \
    .save()


In [ ]:
!apt update && apt install -y netcat
!nc -zv broker 9092



In [2]:
import requests
requests.get("https://es01:9200/", auth=("elastic", "Eselpil2"), verify=False)



/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [1]:
!curl -u elastic:Eselpil2 https://es01:9200 -k


/bin/bash: line 1: curl: command not found


In [2]:
import pyspark
print(pyspark.__version__)


3.5.0


In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.elasticsearch:elasticsearch-spark-30_2.12:8.11.0 pyspark-shell'

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, to_timestamp, when, window, expr
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType

# Créer une session Spark
spark = SparkSession.builder \
    .appName("IoT Kafka") \
    .config("spark.jars.packages",
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,"
            "org.elasticsearch:elasticsearch-spark-30_2.12:8.12.2") \
    .getOrCreate()

# Schéma des données JSON
schema = StructType() \
    .add("machine_id", StringType()) \
    .add("temperature", DoubleType()) \
    .add("humidity", DoubleType()) \
    .add("pressure", DoubleType()) \
    .add("vibration", DoubleType()) \
    .add("latitude", DoubleType()) \
    .add("longitude", DoubleType()) \
    .add("timestamp", StringType())


# Lire depuis Kafka
raw_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:9092") \
    .option("subscribe", "iot_raw_data") \
    .option("startingOffsets", "latest") \
    .load()

# Extraire le JSON
parsed_df = raw_df.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*")

# Conversion du timestamp
parsed_df = parsed_df.withColumn("timestamp", to_timestamp("timestamp"))

# Nettoyage des valeurs manquantes
clean_df = parsed_df.dropna()

# Ajout des règles métiers
alert_df = clean_df.withColumn(
    "status",
    when(col("temperature") > 50, "Alerte")
    .when((col("humidity") < 10) & (col("temperature") > 40), "Critique")
    .when((col("vibration") > 5) & ((col("pressure") < 950) | (col("pressure") > 1050)), "Alerte")
    .otherwise("Normal")
)

# Capteurs inactifs (à traiter séparément en batch)

# Corrélation régionale : moyenne température par région
regional_alerts = alert_df \
    .withColumn("region_lat", expr("round(latitude, 1)")) \
    .withColumn("region_lon", expr("round(longitude, 1)")) \
    .groupBy(window("timestamp", "10 minutes"), "region_lat", "region_lon") \
    .avg("temperature", "humidity") \
    .withColumnRenamed("avg(temperature)", "avg_temp") \
    .withColumnRenamed("avg(humidity)", "avg_humidity") \
    .withColumn("regional_status", when(col("avg_temp") > 45, "Alerte regionale").otherwise("OK"))

# Fonction pour écrire dans Elasticsearch
def write_to_es(batch_df, batch_id):
    print(f"💡 Traitement du batch ID: {batch_id}, {batch_df.count()} lignes")
    
    if batch_df.isEmpty():
        print("⚠️ Batch vide")
        return

    batch_df.write \
        .format("org.elasticsearch.spark.sql") \
        .option("es.nodes", "es01") \
        .option("es.port", "9200") \
        .option("es.nodes.wan.only", "true") \
        .option("es.net.ssl", "true") \
        .option("es.net.ssl.cert.allow.self.signed", "true") \
        .option("es.net.http.auth.user", "elastic") \
        .option("es.net.http.auth.pass", "Eselpil2") \
        .option("es.resource", "agritech_iot_data_enriched/") \
        .mode("append") \
        .save()

# Écriture vers Elasticsearch via foreachBatch
es_query = alert_df.writeStream \
    .outputMode("append") \
    .foreachBatch(write_to_es) \
    .option("checkpointLocation", "/tmp/checkpoint_iot") \
    .start()

# Debug Kafka côté console
debug_query = raw_df.selectExpr("CAST(value AS STRING)").writeStream \
    .format("console") \
    .outputMode("append") \
    .start()

es_query.awaitTermination()
debug_query.awaitTermination()


💡 Traitement du batch ID: 17, 1 lignes
💡 Traitement du batch ID: 18, 1 lignes
💡 Traitement du batch ID: 19, 1 lignes
💡 Traitement du batch ID: 20, 1 lignes
💡 Traitement du batch ID: 21, 1 lignes
💡 Traitement du batch ID: 22, 1 lignes
💡 Traitement du batch ID: 23, 1 lignes


StreamingQueryException: Query [id = 2aa0f8e3-ce5f-4145-9e85-31a0ae6bdce4, runId = faba95f1-c8ac-4d31-b1f8-3ba04678aa4a] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 272, in call
    raise e
  File "/usr/local/spark/python/pyspark/sql/utils.py", line 269, in call
    self.func(DataFrame(jdf, self.session), batch_id)
  File "/tmp/ipykernel_3311/395532143.py", line 87, in write_to_es
    .save()
  File "/usr/local/spark/python/pyspark/sql/readwriter.py", line 966, in save
    self._jwrite.save()
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/usr/local/spark/python/pyspark/sql/utils.py"

In [2]:
raw_df.selectExpr("CAST(value AS STRING)").writeStream \
    .format("console") \
    .outputMode("append") \
    .start()
